### Atari Games

In [1]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [1]:
import gymnasium as gym
import cv2
import torch
from torch import nn
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import gymnasium as gym
from tqdm import tqdm
import os

env = gym.make('ALE/SpaceInvaders-v5', frameskip=4)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [1]:
import torch
import torch.nn as nn

class DQN(nn.Module):
    def __init__(self, num_actions):
        super(DQN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        self.fc_block = nn.Sequential(
            nn.Linear(in_features=1024, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=num_actions)
        )

    def forward(self, input):
        x = self.conv_block(input) 
        out = self.fc_block(x)
        return out

### Własna inplemntacja 

In [61]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import cv2
from tqdm.auto import tqdm

class DQLearn:
    def __init__(self, net, env, gamma=0.99, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995,
                 lr=1e-4, batch_size=128, memory_size=10000):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.net = net.to(self.device)
        self.target_net = type(net)(env.action_space.n).to(self.device)
        self.target_net.load_state_dict(self.net.state_dict())
        self.target_net.eval()

        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)

        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()

        self.frame_stack = deque(maxlen=4)

    def preprocess_frame(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (64, 64))
        return frame.astype(np.float32) / 255.0

    def stack_frames(self, frame, is_new_episode):
        frame = self.preprocess_frame(frame)
        if is_new_episode:
            self.frame_stack = deque([np.zeros((64, 64), dtype=np.float32)] * 4, maxlen=4)
            for _ in range(4):
                self.frame_stack.append(frame)
        else:
            self.frame_stack.append(frame)

        stacked_state = np.stack(self.frame_stack, axis=0)  # shape: (4, 64, 64)
        return stacked_state

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return self.env.action_space.sample()
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
        q_values = self.net(state_tensor)
        return torch.argmax(q_values, dim=1).item()

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train_step(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(self.device)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(self.device)

        q_values = self.net(states).gather(1, actions)
        with torch.no_grad():
            next_q = self.target_net(next_states).max(1)[0].unsqueeze(1)
            q_target = rewards + (1 - dones) * self.gamma * next_q

        loss = self.loss_fn(q_values, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_net(self):
        self.target_net.load_state_dict(self.net.state_dict())

    def train(self, episodes=500, update_target_every=10):
        for ep in tqdm(range(episodes)):
            frame = self.env.reset()[0]
            state = self.stack_frames(frame, is_new_episode=True)
            total_reward = 0

            done = False
            while not done:
                action = self.select_action(state)
                next_frame, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                next_state = self.stack_frames(next_frame, is_new_episode=False)

                self.store_transition(state, action, reward, next_state, done)
                self.train_step()
                state = next_state
                total_reward += reward

            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            if ep % update_target_every == 0:
                self.update_target_net()

            print(f"Episode {ep} | Total Reward: {total_reward:.2f} | Epsilon: {self.epsilon:.3f}")

In [62]:
dql = DQLearn(net=DQN(env.action_space.n), env=env)

In [63]:
dql.train()

  0%|          | 1/500 [03:14<26:59:32, 194.73s/it]

Episode 0 | Total Reward: 10.00 | Epsilon: 0.995


  0%|          | 2/500 [08:01<34:27:59, 249.16s/it]

Episode 1 | Total Reward: 45.00 | Epsilon: 0.990


  0%|          | 2/500 [08:08<33:49:13, 244.49s/it]


KeyboardInterrupt: 

### Szkoenie modelu za pomoca PPO z stable_baselines

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import gymnasium as gym
from tqdm import tqdm
import os

os.makedirs("checkpoints", exist_ok=True)

env = gym.make("ALE/SpaceInvaders-v5")
env = Monitor(env)

env = DummyVecEnv([lambda: env])

env = VecFrameStack(env, n_stack=4)

model = DQN(
    "CnnPolicy",
    env,
    buffer_size=50_000, 
    verbose=1
)

total_timesteps = 1_000_000
epoch_size = 1000
num_epochs = total_timesteps // epoch_size

for epoch in tqdm(range(1, num_epochs + 1), desc="Training Progress", ncols=100):
    model.learn(total_timesteps=epoch_size, reset_num_timesteps=False)
    
    if epoch % 50 == 0:
        model.save(f"checkpoints/ppo_spaceinvaders_epoch_{epoch}")

model.save("ppo_spaceinvaders_final")


In [ ]:
os.makedirs("checkpoints", exist_ok=True)

env = gym.make("ALE/SpaceInvaders-v5")
env = Monitor(env)

env = DummyVecEnv([lambda: env])

env = VecFrameStack(env, n_stack=4)

model = DQN("CnnPolicy", env, buffer_size=30000, verbose=1)

total_timesteps = 1_000_000
epoch_size = 1000
num_epochs = total_timesteps // epoch_size

for epoch in tqdm(range(1, num_epochs + 1), desc="Training Progress", ncols=100):
    model.learn(total_timesteps=epoch_size, reset_num_timesteps=False)
    
    if epoch % 50 == 0:
        model.save(f"checkpoints/ppo_spaceinvaders_epoch_{epoch}")

model.save("ppo_spaceinvaders_final")


----------------------------------
Training Progress:  91%|███████████████████████████████████▎   | 907/1000 [1:59:54<13:03,  8.43s/it]
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 507      |
|    ep_rew_mean      | 329      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 1828     |
|    fps              | 120      |
|    time_elapsed     | 2        |
|    total_timesteps  | 907292   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.628    |
|    n_updates        | 214322   |
----------------------------------

In [3]:
env = gym.make("ALE/SpaceInvaders-v5")
env = Monitor(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)

model = DQN.load("/home/plorenc/Desktop/AiR_ISS/ML/RL/checkpoints/ppo_spaceinvaders_epoch_10.zip", env=env)

total_timesteps = 1_000_000
epoch_size = 1000
num_epochs = total_timesteps // epoch_size

for epoch in tqdm(range(1, 1000 + 1), desc="Training Progress", ncols=100):
    model.learn(total_timesteps=epoch_size, reset_num_timesteps=False)
    
    if epoch % 50 == 0:
        model.save(f"checkpoints/ppo_spaceinvaders_epoch_{epoch}")

model.save("ppo_spaceinvaders_final")


model.save("checkpoints/ppo_spaceinvaders_epoch_150")


Training Progress:  46%|█████████████████                    | 460/1000 [1:05:14<1:19:18,  8.81s/it]

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 550      |
|    ep_rew_mean      | 360      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2820     |
|    fps              | 118      |
|    time_elapsed     | 2        |
|    total_timesteps  | 1470296  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.796    |
|    n_updates        | 355073   |
----------------------------------


Training Progress:  46%|█████████████████                    | 462/1000 [1:05:31<1:17:49,  8.68s/it]

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 555      |
|    ep_rew_mean      | 368      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2824     |
|    fps              | 114      |
|    time_elapsed     | 6        |
|    total_timesteps  | 1472692  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.32     |
|    n_updates        | 355672   |
----------------------------------


Training Progress:  46%|█████████████████▏                   | 463/1000 [1:05:40<1:17:58,  8.71s/it]

...

Training Progress: 100%|██████████████████████████████████████▉| 997/1000 [2:19:45<00:24,  8.33s/it]

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 583      |
|    ep_rew_mean      | 400      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3800     |
|    fps              | 119      |
|    time_elapsed     | 0        |
|    total_timesteps  | 2007035  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 3.09     |
|    n_updates        | 489258   |
----------------------------------


Training Progress: 100%|██████████████████████████████████████▉| 999/1000 [2:20:02<00:08,  8.36s/it]

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 584      |
|    ep_rew_mean      | 400      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 3804     |
|    fps              | 118      |
|    time_elapsed     | 3        |
|    total_timesteps  | 2009391  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.469    |
|    n_updates        | 489847   |
----------------------------------


Training Progress: 100%|██████████████████████████████████████| 1000/1000 [2:20:11<00:00,  8.41s/it]


In [ ]:
from stable_baselines3 import DQN 
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import gymnasium as gym

# Tworzenie środowiska z renderowaniem
env = gym.make("ALE/SpaceInvaders-v5", render_mode="human")
env = Monitor(env)

# Konieczne opakowania
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)

# Załaduj model
model = DQN.load("/home/plorenc/Desktop/AiR_ISS/ML/RL/checkpoints/ppo_spaceinvaders_epoch_750.zip", env=env)

# Uruchomienie agenta
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()


Wrapping the env in a VecTransposeImage.


/home/plorenc/Desktop/venv-y/RL venv/lib/python3.11/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 24.19GB > 3.64GB
  warnings.warn(


KeyboardInterrupt: 

: 

### Optymalizacja paramterow podczas uczenia z optuna

In [ ]:
import os
import gymnasium as gym
import optuna
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from tqdm import tqdm

# Stwórz foldery
os.makedirs("optuna_checkpoints", exist_ok=True)

# Funkcja tworząca środowisko
def make_env():
    return Monitor(gym.make("ALE/SpaceInvaders-v5"))

# Funkcja celu dla Optuny
def optimize_ppo(trial):
    env = make_env()

    # Przestrzeń hiperparametrów do przeszukania
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    gamma = trial.suggest_uniform('gamma', 0.9, 0.9999)
    n_steps = trial.suggest_categorical('n_steps', [128, 256, 512, 1024])
    ent_coef = trial.suggest_loguniform('ent_coef', 0.00000001, 0.1)
    clip_range = trial.suggest_uniform('clip_range', 0.1, 0.4)
    gae_lambda = trial.suggest_uniform('gae_lambda', 0.8, 1.0)

    model = PPO(
        "CnnPolicy",
        env,
        learning_rate=learning_rate,
        gamma=gamma,
        n_steps=n_steps,
        ent_coef=ent_coef,
        clip_range=clip_range,
        gae_lambda=gae_lambda,
        verbose=0,
    )

    model.learn(total_timesteps=100_000)
    
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5, render=False)

    return mean_reward

study = optuna.create_study(direction='maximize')
study.optimize(optimize_ppo, n_trials=20)  

print("✅ Najlepsze parametry:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

best_params = study.best_params
env = make_env()
best_model = PPO(
    "CnnPolicy",
    env,
    **best_params,
    verbose=1,
)

total_timesteps = 1_000_000
epoch_size = 10000
num_epochs = total_timesteps // epoch_size

for epoch in tqdm(range(1, num_epochs + 1), desc="Training (best params)", ncols=100):
    best_model.learn(total_timesteps=epoch_size, reset_num_timesteps=False)

    if epoch % 50 == 0:
        best_model.save(f"optuna_checkpoints/ppo_best_epoch_{epoch}")

best_model.save("ppo_spaceinvaders_best_final")


[I 2025-06-12 21:04:37,928] Trial 16 finished with value: 285.0 and parameters: {'learning_rate': 2.965437188017218e-05, 'gamma': 0.9787983489982238, 'n_steps': 1024, 'ent_coef': 0.00477870995327333, 'clip_range': 0.1935358383718346, 'gae_lambda': 0.8044171446097719}. Best is trial 4 with value: 288.0.


[I 2025-06-12 21:12:26,854] Trial 17 finished with value: 139.0 and parameters: {'learning_rate': 0.00011701857799450871, 'gamma': 0.9272375397750325, 'n_steps': 128, 'ent_coef': 1.9209513067547926e-07, 'clip_range': 0.27323501778012177, 'gae_lambda': 0.9396958849567045}. Best is trial 4 with value: 288.0.


[I 2025-06-12 21:20:23,110] Trial 18 finished with value: 285.0 and parameters: {'learning_rate': 1.0306674375625183e-05, 'gamma': 0.9537869744560241, 'n_steps': 1024, 'ent_coef': 1.7426922520679073e-08, 'clip_range': 0.2060662947543671, 'gae_lambda': 0.8979545255068435}. Best is trial 4 with value: 288.0.


[I 2025-06-12 21:28:14,208] Trial 19 finished with value: 297.0 and parameters: {'learning_rate': 0.0003603034714510459, 'gamma': 0.9746210073550151, 'n_steps': 256, 'ent_coef': 5.510438562118013e-05, 'clip_range': 0.14403864725593685, 'gae_lambda': 0.8416304787950954}. Best is trial 19 with value: 297.0.

